In [1]:
import os
import json
from dotenv import load_dotenv
from IPython.display import display, Markdown, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good")
else:
    print("There might be a problem with the API key")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good


In [4]:
links = fetch_website_links('https://huggingface.co')
links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/MiniMaxAI/MiniMax-M2.1',
 '/tencent/HY-MT1.5-1.8B',
 '/zai-org/GLM-4.7',
 '/Qwen/Qwen-Image-2512',
 '/LGAI-EXAONE/K-EXAONE-236B-A23B',
 '/models',
 '/spaces/Wan-AI/Wan2.2-Animate',
 '/spaces/mrfakename/Z-Image-Turbo',
 '/spaces/microsoft/TRELLIS.2',
 '/spaces/selfit-camera/Omni-Image-Editor',
 '/spaces/prithivMLmods/Qwen-Image-Edit-2511-LoRAs-Fast',
 '/spaces',
 '/datasets/facebook/research-plan-gen',
 '/datasets/bigai/TongSIM-Asset',
 '/datasets/bshada/open-schematics',
 '/datasets/nebius/SWE-rebench-openhands-trajectories',
 '/datasets/gaia-benchmark/GAIA',
 '/datasets',
 '/join',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/inference/models',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly

In [5]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(url):
    user_prompt="""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):
"""

    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [7]:
print(get_links_user_prompt('https://huggingface.co'))


Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):
/
/models
/datasets
/spaces
/docs
/enterprise
/pricing
/login
/join
/spaces
/models
/MiniMaxAI/MiniMax-M2.1
/tencent/HY-MT1.5-1.8B
/zai-org/GLM-4.7
/Qwen/Qwen-Image-2512
/LGAI-EXAONE/K-EXAONE-236B-A23B
/models
/spaces/Wan-AI/Wan2.2-Animate
/spaces/mrfakename/Z-Image-Turbo
/spaces/microsoft/TRELLIS.2
/spaces/selfit-camera/Omni-Image-Editor
/spaces/prithivMLmods/Qwen-Image-Edit-2511-LoRAs-Fast
/spaces
/datasets/facebook/research-plan-gen
/datasets/bigai/TongSIM-Asset
/datasets/bshada/open-schematics
/datasets/nebius/SWE-rebench-openhands-trajectories
/datasets/gaia-benchmark/GAIA
/datasets
/join
/enterprise
/enterprise
/enterprise
/enterprise
/enterprise
/enterprise
/enterprise
/inference/models
/pricing#endpoints

In [10]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [11]:
select_relevant_links('https://huggingface.co')

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'docs', 'url': 'https://huggingface.co/docs'},
  {'type': 'product page', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'Discord', 'url': 'https://huggingface.co/join/discord'},
  {'type': 'Community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'Status page', 'url': 'https://status.huggingface.co/'}]}